## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-05-14-51-51 +0000,nyt,2 Spooky Flashes Lit Up the Moon. Here’s What ...,https://www.nytimes.com/2025/11/05/science/moo...
1,2025-11-05-14-47-48 +0000,bbc,'We are worried some families may not afford 2...,https://www.bbc.com/news/articles/c781v0w2q7do...
2,2025-11-05-14-47-14 +0000,cbc,Japan sends in the military to deal with surge...,https://www.cbc.ca/news/world/japan-bears-mili...
3,2025-11-05-14-46-41 +0000,nyt,"Tariffs Are Here to Stay, Even if the Supreme ...",https://www.nytimes.com/2025/11/05/us/politics...
4,2025-11-05-14-44-19 +0000,nyt,"Shutdown Becomes Longest in History, as Trump ...",https://www.nytimes.com/2025/11/04/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,52
160,new,33
48,mamdani,29
200,election,21
157,will,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
391,2025-11-04-17-18-25 +0000,cbc,"New York City's new mayor, Zohran Mamdani, cal...",https://www.cbc.ca/news/world/new-york-city-el...,243
169,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...,227
37,2025-11-05-14-02-20 +0000,nypost,Socialist NYC Mayor-elect Zohran Mamdani fires...,https://nypost.com/2025/11/05/us-news/zohran-m...,175
189,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...,169
160,2025-11-05-07-26-00 +0000,wsj,Zohran Mamdani sealed his mayoral victory in N...,https://www.wsj.com/politics/elections/the-vot...,151


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
391,243,2025-11-04-17-18-25 +0000,cbc,"New York City's new mayor, Zohran Mamdani, cal...",https://www.cbc.ca/news/world/new-york-city-el...
169,84,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...
8,77,2025-11-05-14-42-00 +0000,wsj,Lawmakers are hopeful they can reach a biparti...,https://www.wsj.com/politics/policy/lawmakers-...
189,62,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...
163,56,2025-11-05-07-06-46 +0000,startribune,"Kaohly Her wins historic election, will be St....",https://www.startribune.com/st-paul-mayor-elec...
23,49,2025-11-05-14-18-00 +0000,wsj,UPS Cargo Plane Crash in Kentucky Leaves at Le...,https://www.wsj.com/us-news/ups-cargo-plane-cr...
277,49,2025-11-04-23-11-18 +0000,cbc,Democratic candidates win key governor races a...,https://www.cbc.ca/news/world/virginia-new-jer...
342,49,2025-11-04-20-01-11 +0000,cbc,How budget day unfolded in Ottawa — from budge...,https://www.cbc.ca/news/politics/livestory/how...
100,49,2025-11-05-11-00-00 +0000,nypost,"Eric Trump shares his dad’s family rules, talk...",https://nypost.com/2025/11/05/us-news/eric-tru...
343,42,2025-11-04-19-57-43 +0000,nypost,Palantir CEO Alex Karp blasts ‘Big Short’ inve...,https://nypost.com/2025/11/04/business/palanti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
